## Plotting Azimuth axis on StarTracker images - 05-Dec-22

Craig Lage

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
%matplotlib inline

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

path = '/project/GenericCamera/101/2022/11/22/'

In [ ]:
# Get the image and solve for the WCS
seqNum = 39
filename = path + f'GC101_O_20221122_{seqNum:06}.fits'
exp = starTrackerFileToExposure(filename)
results = runCharactierizeImage(exp, 5, 25)
solverResults = solver.run(exp, results.sourceCat, isWideField=True)
newWcs = solverResults.wcs
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

In [ ]:
DIR = "/project/GenericCamera/101/2022/11/22/"
HEAD = "GC101_O_20221122_"
hdulist = pf.open(f"{DIR}{HEAD}0000{seqNum}.fits")
header = hdulist[0].header
time = Time(header["DATE-OBS"])

Alt = 90.0
Az = 180.0
now = Time(datetime.now())
altAz = SkyCoord(AltAz(alt=Alt*u.deg, az=Az*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz.transform_to(ICRS)
spherePoint = SpherePoint(afwAngle(RaDec.ra.rad), afwAngle(RaDec.dec.rad))
point = newWcs.skyToPixel(spherePoint)
print(point)

In [ ]:
azimuth_axis = (1560.85, 1257.151) # From Adrian Shestakov - 10Dec22
newWcs.getPixelOrigin().x, newWcs.getPixelOrigin().y
separation_red_green = np.sqrt((point.x - azimuth_axis[0])**2 + (point.y - azimuth_axis[1])**2)
separation_red_green *= newWcs.getPixelScale().asArcseconds()
separation_red_yellow = np.sqrt((point.x - newWcs.getPixelOrigin().x)**2 + (point.y - newWcs.getPixelOrigin().y)**2)
separation_red_yellow *= newWcs.getPixelScale().asArcseconds()
print(separation_red_green, separation_red_yellow)

In [ ]:
seqNum = 43
filename = path + f'GC101_O_20221122_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)

Ncenter = (300, 1800)
Nlength = 500.0
Nlabel = 650.0


# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field 2022112200043")
img = plt.imshow(exp.image.array,  vmin=1, vmax=6000, interpolation='Nearest', cmap='gray')
plt.scatter(point.x, point.y, marker='+', color='green', s=200,lw=2)
plt.text(point.x+200, point.y, "Astronomical Zenith", color='green', fontsize=18)
plt.scatter(azimuth_axis[0], azimuth_axis[1], marker='+', color='red', s=200,lw=2)
plt.text(azimuth_axis[0]-1000, azimuth_axis[1], "Azimuth Axis", color='red', fontsize=18)
plt.scatter(newWcs.getPixelOrigin().x, newWcs.getPixelOrigin().y, marker='+', color='yellow', s=200,lw=2)
plt.text(newWcs.getPixelOrigin().x-100,newWcs.getPixelOrigin().x-700, "Image center", color='yellow', fontsize=18)

plt.text(azimuth_axis[0]-1500, azimuth_axis[1]+500, f"Separation Red_Green = {separation_red_green:.2f} arcseconds", color='red', fontsize=18)
plt.text(azimuth_axis[0]-1500, azimuth_axis[1]+700, f"Separation Red_Yellow = {separation_red_yellow:.2f} arcseconds", color='red', fontsize=18)
xs = []
ys = []
r = 1480.0
for deg in range(10, 58):
    x = azimuth_axis[0] + r * np.cos(deg * np.pi / 180.0)
    y = azimuth_axis[1] - r * np.sin(deg * np.pi / 180.0)
    xs.append(x)
    ys.append(y)
plt.plot(xs, ys, color='red')
xs = []
ys = []
r = 1460.0
for deg in range(150, 220):
    x = azimuth_axis[0] + r * np.cos(deg * np.pi / 180.0)
    y = azimuth_axis[1] - r * np.sin(deg * np.pi / 180.0)
    xs.append(x)
    ys.append(y)
plt.plot(xs, ys, color='red')
xs = []
ys = []
r = 850.0
for deg in range(-25, 40):
    x = azimuth_axis[0] + r * np.cos(deg * np.pi / 180.0)
    y = azimuth_axis[1] - r * np.sin(deg * np.pi / 180.0)
    xs.append(x)
    ys.append(y)
plt.plot(xs, ys, color='red')

colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Azimuth_Axis_Center_Wide_20221122_43.pdf")

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

path = '/project/GenericCamera/102/2022/11/22/'

In [ ]:
# Get the image and solve for the WCS
seqNum = 30
filename = path + f'GC102_O_20221122_{seqNum:06}.fits'
exp = starTrackerFileToExposure(filename)
results = runCharactierizeImage(exp, 5, 25)
solverResults = solver.run(exp, results.sourceCat, percentageScaleError=60.0, isWideField=False)
newWcs = solverResults.wcs
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

In [ ]:
DIR = "/project/GenericCamera/102/2022/11/22/"
HEAD = "GC102_O_20221122_"
hdulist = pf.open(f"{DIR}{HEAD}0000{seqNum}.fits")
header = hdulist[0].header
time = Time(header["DATE-OBS"])

Alt = 90.0
Az = 180.0
now = Time(datetime.now())
altAz = SkyCoord(AltAz(alt=Alt*u.deg, az=Az*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz.transform_to(ICRS)
spherePoint = SpherePoint(afwAngle(RaDec.ra.rad), afwAngle(RaDec.dec.rad))
point = newWcs.skyToPixel(spherePoint)
print(point)

In [ ]:
azimuth_axis = (1867.87282572, 974.5819506) # From Adrian Shestakov - 10Dec22
newWcs.getPixelOrigin().x, newWcs.getPixelOrigin().y
separation_red_green = np.sqrt((point.x - azimuth_axis[0])**2 + (point.y - azimuth_axis[1])**2)
separation_red_green *= newWcs.getPixelScale().asArcseconds()
separation_red_yellow = np.sqrt((point.x - newWcs.getPixelOrigin().x)**2 + (point.y - newWcs.getPixelOrigin().y)**2)
separation_red_yellow *= newWcs.getPixelScale().asArcseconds()
print(separation_red_green, separation_red_yellow)

In [ ]:
seqNum = 43
filename = path + f'GC102_O_20221122_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)

Ncenter = (300, 1800)
Nlength = 500.0
Nlabel = 650.0


# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Narrow Field 2022112200043")
img = plt.imshow(exp.image.array,  vmin=1, vmax=2000, interpolation='Nearest', cmap='gray')
plt.scatter(point.x, point.y, marker='+', color='green', s=200,lw=2)
plt.text(point.x-300, point.y-200, "Astronomical Zenith", color='green', fontsize=18)
plt.scatter(azimuth_axis[0], azimuth_axis[1], marker='+', color='red', s=200,lw=2)
plt.text(azimuth_axis[0]-1000, azimuth_axis[1], "Azimuth Axis", color='red', fontsize=18)
plt.scatter(newWcs.getPixelOrigin().x, newWcs.getPixelOrigin().y, marker='+', color='yellow', s=200,lw=2)
plt.text(newWcs.getPixelOrigin().x-200,newWcs.getPixelOrigin().x-500, "Image center", color='yellow', fontsize=18)

plt.text(azimuth_axis[0]-1500, azimuth_axis[1]+500, f"Separation Red_Green = {separation_red_green:.2f} arcseconds", color='red', fontsize=18)
plt.text(azimuth_axis[0]-1500, azimuth_axis[1]+700, f"Separation Red_Yellow = {separation_red_yellow:.2f} arcseconds", color='red', fontsize=18)
xs = []
ys = []
r = 1900.0
for deg in range(200, 245):
    x = azimuth_axis[0] + r * np.cos(deg * np.pi / 180.0)
    y = azimuth_axis[1] - r * np.sin(deg * np.pi / 180.0)
    xs.append(x)
    ys.append(y)
plt.plot(xs, ys, color='red')

xs = []
ys = []
r = 1200.0
for deg in range(300,360):
    x = azimuth_axis[0] + r * np.cos(deg * np.pi / 180.0)
    y = azimuth_axis[1] - r * np.sin(deg * np.pi / 180.0)
    xs.append(x)
    ys.append(y)
plt.plot(xs, ys, color='red')
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Azimuth_Axis_Center_Narrow_20221122_43.pdf")